![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [233]:
# We've loaded your first package for you! You can add as many cells as you need.
import numpy as np
import pandas as pd

#1. Importing the data
prices = pd.read_csv('data/airbnb_price.csv')
room_types = pd.read_excel('data/airbnb_room_type.xlsx')
reviews = pd.read_csv('data/airbnb_last_review.tsv',sep = '\t')

In [234]:
#2. Cleaning the price column
prices['price'] = prices['price'].str.rstrip(to_strip = " dollars")
prices['price'] = pd.to_numeric(prices['price'])

In [235]:
#3. Calculating average price (without outlier prices)
prices = prices[prices['price']>0]
avg_price = prices['price'].mean().round(decimals = 2)
print("Avg price is",avg_price)

Avg price is 141.82


In [236]:
#4. Comparing costs to the private rental market
prices['price_per_month'] = prices['price']*365/12
average_price_per_month = prices['price_per_month'].mean().round(decimals = 2)
difference = (average_price_per_month - 3100).round(decimals = 2)
print("Airbnb vs Mkt difference:",difference)

Airbnb vs Mkt difference: 1213.61


In [237]:
#5. Cleaning the room_type column
room_types['room_type'] = room_types['room_type'].str.lower()
room_types["room_type"] = room_types["room_type"].astype("category")
room_frequencies = room_types['room_type'].value_counts(normalize = False)

print(room_frequencies)

entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64


In [238]:
#6. What timeframe are we working with?
reviews['last_review'] = pd.to_datetime(reviews['last_review'])

first_reviewed = reviews['last_review'].dt.date.min()
last_reviewed = reviews['last_review'].dt.date.max()

print('Oldest review date:',first_reviewed)
print('Most recent review date:',last_reviewed)

Oldest review date: 2019-01-01
Most recent review date: 2019-07-09


In [239]:
#7. Joining the DataFrames
rooms_and_prices = prices.merge(room_types, how='outer', on = 'listing_id')
airbnb_merged = rooms_and_prices.merge(reviews, how='outer', on = 'listing_id')
airbnb_merged.dropna(inplace = True)

print('Duplicated values:',airbnb_merged.duplicated().sum())

Duplicated values: 0


In [240]:
#8. Analyzing listing prices by NYC borough
airbnb_merged['borough'] = airbnb_merged['nbhood_full'].str.partition(sep = ',')[0]

boroughs  = airbnb_merged.groupby('borough')['price'].agg(['sum', 'mean', 'median',  'count'])
boroughs = boroughs.round(decimals = 2).sort_values('mean',ascending=False)

print(boroughs)


                     sum    mean  median  count
borough                                        
Manhattan      1898417.0  184.04   149.0  10315
Brooklyn       1275250.0  122.02    95.0  10451
Queens          320715.0   92.83    70.0   3455
Staten Island    22974.0   86.04    71.0    267
Bronx            55156.0   79.25    65.0    696


In [241]:
#9. Price range by borough
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

airbnb_merged['price range'] = pd.cut(x=airbnb_merged['price'], bins =[0,69,175,350,np.inf], labels = label_names)

prices_by_borough = airbnb_merged.groupby(['borough','price range'])['price range'].count()
print(prices_by_borough)

borough        price range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price range, dtype: int64


In [242]:
#10. Storing the final result
airbnb_analysis = {'avg_price':avg_price, 
                   'average_price_per_month':average_price_per_month,
                   'difference':difference,
                   'room_frequencies':room_frequencies,
                   'first_reviewed':first_reviewed,
                   'last_reviewed':last_reviewed, 
                   'prices_by_borough':prices_by_borough}

print(airbnb_analysis)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  